In [1]:
%run ipy_setup.py
%matplotlib inline

/home/neffra/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [28]:
signature_mycbp2 = "/home/neffra/zhang-ipy/MYCBP2_signature.txt"
upreg_genes = set()
downreg_genes = set()
with open(signature_mycbp2, 'r') as fp:
    for line in fp: 
        splitline = line.split("\t")
        if (splitline[0].split('.')[0] == "sh"):
            fdr = "0." + splitline[0].split('.')[-2]
            if fdr == "0.1":
                if (splitline[0].split('.')[-1] == "up"):
                    upreg_genes = upreg_genes.union(set(splitline[1:]))
                elif (splitline[0].split('.')[-1] == "dn"):    
                    downreg_genes = downreg_genes.union(set(splitline[1:]))
upreg_genes_mycbp2 = upreg_genes.difference(downreg_genes)
downreg_genes_mycbp2 = downreg_genes.difference(upreg_genes)

In [29]:
signature_mtor = "/home/neffra/minerva/MTOR_signature.txt"
upreg_genes = set()
downreg_genes = set()
with open(signature_mtor, 'r') as fp:
    for line in fp: 
        splitline = line.split("\t")
        if (splitline[0].split('.')[0] == "sh"):
            fdr = "0." + splitline[0].split('.')[-2]
            if fdr == "0.05":
                if (splitline[0].split('.')[-1] == "up"):
                    upreg_genes = upreg_genes.union(set(splitline[1:]))
                elif (splitline[0].split('.')[-1] == "dn"):    
                    downreg_genes = downreg_genes.union(set(splitline[1:]))
upreg_genes_mtor = upreg_genes.difference(downreg_genes)
downreg_genes_mtor = downreg_genes.difference(upreg_genes)

In [40]:
alias_to_gene = pd.read_csv("/media/sf_dd/zhang/alias_to_gene.txt", sep =" ", header=None)
def forward_translate(x):
    if x in alias_to_gene[0]:
        return alias_to_gene[alias_to_gene[0]==x][1]
    else:
        return None
def reverse_translate(x):
    if x in alias_to_gene[1]:
        return alias_to_gene[alias_to_gene[1]==x][0]
    else:
        return None
def lookup(x):
    if type(x) != type("string"):
        return x
    rev = reverse_translate(x)
    if rev:
        return x
    fwd = forward_translate(x)
    if fwd:
        return str(fwd)
    else:
        return x

In [55]:
directory = "/sc/orga/projects/zhangb03a/neffr01/tcga_brca_table_data/"
filen = "tcga_erpos_adjnorm_deg_limma.txt"
bnfile = "u4_bn_keydrivers.txt"
bn = pd.read_csv(directory + bnfile, sep="\t")
tcga_rna_DEG = pd.read_csv(directory + filen, sep="\t")
tcga_rna_DEG["gene"] = tcga_rna_DEG["gene"].map(lambda x: lookup(x))
bn["gene"] = bn["gene"].map(lambda x: lookup(x))

import networkx as nx
graph = nx.DiGraph()

network_tcga = "/home/neffra/minerva/brca_files/tcga_erpos__MEGENA_Network.txt"
network_set = pd.read_csv(network_tcga, sep="\t")
pbar = ProgressBar(len(network_set))
for ix, row in network_set.iterrows():
        pbar.animate()
        firstg = lookup(row["row"].split("|")[0])
        secondg = lookup(row["col"].split("|")[0])
        graph.add_edge(firstg, secondg, weight=row["weight"])
pbar = ProgressBar(len(graph))
for node in graph.nodes_iter():
    pbar.animate()
    expr = tcga_rna_DEG[tcga_rna_DEG["gene"]==node][tcga_rna_DEG["P.Value"]<0.01]["logFC"].values
    expr = expr[0] if len(expr) != 0 else 0
    expr = round(expr, 2)
    if expr >= 0.1:
        expr = 1
    elif -1*expr >= 0.1:
        expr = -1
    else:
        expr = 0
    graph.node[node]["expression"] = expr
    graph.node[node]["keydriver"] = 1 if node in set(bn["gene"]) else 0
    if node in upreg_genes2:
        graph.node[node]["mycbp2sig"] = 1
    elif node in downreg_genes2:
        graph.node[node]["mycbp2sig"] = -1
    else:
        graph.node[node]["mycbp2sig"] = 0

[*****************56%*                 ]  29997 of 53405 complete Elapsed: 0.1 minutes	 Remaining: 0.0 minutes

/home/neffra/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


[*****************94%****************  ]  18435 of 19654 complete Elapsed: 0.5 minutes	 Remaining: 0.0 minutes


In [56]:
nx.write_gml(graph, "/media/sf_dd/zhang/megena_with_expr_mycbp2_sig_10pct.gml")

In [49]:
directory = "/sc/orga/projects/zhangb03a/neffr01/tcga_brca_table_data/"
filen = "tcga_erpos_adjnorm_deg_limma.txt"
bnfile = "u4_bn_keydrivers.txt"
bn = pd.read_csv(directory + bnfile, sep="\t")
tcga_rna_DEG = pd.read_csv(directory + filen, sep="\t")
tcga_rna_DEG["gene"] = tcga_rna_DEG["gene"].map(lambda x: lookup(x))
bn["gene"] = bn["gene"].map(lambda x: lookup(x))

import networkx as nx
graph = nx.DiGraph()

network_tcga = "/home/neffra/zhang-ipy/brca/union4BNs.ERpos.txt"
network_set = pd.read_csv(network_tcga, sep=" ")
pbar = ProgressBar(len(network_set))
for ix, row in network_set.iterrows():
        pbar.animate()
        firstg = lookup(row["src"])
        secondg = lookup(row["dst"])
        graph.add_edge(firstg, secondg)
pbar = ProgressBar(len(graph))
for node in graph.nodes_iter():
    pbar.animate()
    expr = tcga_rna_DEG[tcga_rna_DEG["gene"]==node][tcga_rna_DEG["P.Value"]<0.01]["logFC"].values
    expr = expr[0] if len(expr) != 0 else 0
    expr = round(expr, 2)
    if expr >= 0.1:
        expr = 1
    elif -1*expr >= 0.1:
        expr = -1
    else:
        expr = 0
    graph.node[node]["expression"] = expr
    graph.node[node]["keydriver"] = 1 if node in set(bn["gene"]) else 0
    if node in upreg_genes2:
        graph.node[node]["mycbp2sig"] = 1
    elif node in downreg_genes2:
        graph.node[node]["mycbp2sig"] = -1
    else:
        graph.node[node]["mycbp2sig"] = 0

[*****************67%*****             ]  50778 of 76134 complete Elapsed: 0.1 minutes	 Remaining: 0.0 minutes

/home/neffra/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


[*****************91%***************   ]  18970 of 20814 complete Elapsed: 0.5 minutes	 Remaining: 0.0 minutes


In [50]:
nx.write_gml(graph, "/media/sf_dd/zhang/bn_with_expr_mycbp2_sig.gml")

## Module Enrichment

In [57]:
tcga_modules = dict()
tcga_mod_file = "/home/neffra/minerva/brca_files/tcga_erpos__multiscale_significant.mapped_modules.txt"
with open(tcga_mod_file, 'r') as fp:
    for line in fp:
        l = [lookup(a.split("|")[0]) for a in line.strip().split("\t")]
        tcga_modules[l[0]] = {"members":set(l[1:]), "hubs":set()}

In [80]:
modules_with_muts = []
pbar = ProgressBar(len(tcga_modules))
for name, module in tcga_modules.iteritems():
    pbar.animate()
    if "MTOR" in module["members"]:
        print name
    modules_with_muts.append([name, len(downreg_genes2.intersection(module["members"])), len(upreg_genes2.intersection(module["members"])), len(module["members"])])

comp1_74
comp1_301
comp1_6


In [72]:
list1 = sorted(modules_with_muts, key=lambda x:x[2]/(x[3]+.1), reverse=True)
list1

[['comp1_589', 0, 8, 13],
 ['comp1_566', 0, 6, 11],
 ['comp1_554', 0, 7, 13],
 ['comp1_382', 1, 8, 15],
 ['comp1_693', 0, 18, 35],
 ['comp1_257', 0, 8, 18],
 ['comp1_520', 1, 7, 16],
 ['comp1_641', 2, 13, 30],
 ['comp1_108', 2, 24, 56],
 ['comp1_728', 2, 6, 15],
 ['comp1_556', 0, 4, 10],
 ['comp1_383', 1, 16, 41],
 ['comp1_373', 3, 5, 13],
 ['comp1_372', 2, 8, 21],
 ['comp1_612', 1, 4, 11],
 ['comp1_359', 2, 13, 36],
 ['comp1_750', 0, 8, 23],
 ['comp1_286', 3, 16, 47],
 ['comp1_640', 5, 6, 18],
 ['comp1_620', 2, 5, 15],
 ['comp1_587', 0, 4, 12],
 ['comp1_594', 0, 4, 12],
 ['comp1_170', 1, 4, 12],
 ['comp1_487', 2, 25, 79],
 ['comp1_535', 7, 6, 19],
 ['comp1_243', 4, 5, 16],
 ['comp1_793', 2, 4, 13],
 ['comp1_66', 5, 20, 66],
 ['comp1_339', 1, 3, 10],
 ['comp1_290', 2, 3, 10],
 ['comp1_254', 2, 3, 10],
 ['comp1_252', 2, 3, 10],
 ['comp1_549', 0, 5, 17],
 ['comp1_561', 3, 4, 14],
 ['comp1_521', 1, 5, 18],
 ['comp1_573', 5, 12, 44],
 ['comp1_267', 1, 6, 22],
 ['comp1_723', 2, 3, 11],
 ['c

In [84]:
mtor_related = '''PIK3CA PIK3R1 PIK3R2 PTEN PDPK1 AKT1 AKT2 FOXO1 FOXO3 MTOR RICTOR TSC1 TSC2 RHEB AKT1S1 RPTOR MLST8 CHEK1'''.split(" ")

In [85]:
for i in mtor_related:
    print i, i in downreg_genes2, i in upreg_genes2

PIK3CA False False
PIK3R1 False False
PIK3R2 True False
PTEN False True
PDPK1 False False
AKT1 False True
AKT2 False False
FOXO1 False False
FOXO3 False False
MTOR False False
RICTOR False False
TSC1 True False
TSC2 False True
RHEB False True
AKT1S1 False False
RPTOR False False
MLST8 True False
CHEK1 True False


In [ ]:
tcga_modules

In [81]:
tcga_modules["comp1_301"]

{'hubs': set(),
 'members': {'ARHGEF37',
  'ARID1A',
  'ATP13A2',
  'CLCN6',
  'DFFA',
  'DHDDS',
  'ECE1',
  'EIF2C1',
  'EIF2C4',
  'EPS15',
  'FUBP1',
  'GNB1',
  'HIVEP3',
  'HNRNPR',
  'HPRT1',
  'KIAA0090',
  'KIAA0467',
  'KIAA0494',
  'KIAA0495',
  'KIAA0562',
  'KIF1B',
  'LDLRAP1',
  'LZIC',
  'MACF1',
  'MAN1C1',
  'MFN2',
  'MIER1',
  'MKNK1',
  'MOBKL2C',
  'MTHFR',
  'MTOR',
  'MUL1',
  'NBPF1',
  'NDST1',
  'NIPAL3',
  'NMNAT1',
  'NOL9',
  'OSBPL9',
  'OTUD3',
  'PINK1',
  'PLAC1',
  'PRDM2',
  'PUM1',
  'RERE',
  'SEPN1',
  'SLC35D1',
  'SLC9A1',
  'SPEN',
  'SPTAN1',
  'THRAP3',
  'UBE4B',
  'UBIAD1',
  'UBR4',
  'USP24',
  'VPS13D',
  'WDTC1',
  'ZBTB40',
  'ZMYM4',
  'ZYG11B'}}

In [103]:
pd.DataFrame([(a,"up") for a in list(upreg_genes2)], columns=["Gene","module"]).astype(str).to_csv("/sc/orga/projects/zhangb03a/neffr01/Enrichment_tcga_2/core_BRCA.txt", sep="\t", index=None)

## MTOR vs MYCBP2 signatures

In [18]:
len(downreg_genes_mtor.intersection(downreg_genes_mycbp2)), len(downreg_genes_mtor), len(downreg_genes_mycbp2)

(382, 1745, 1692)

In [19]:
len(upreg_genes_mtor.intersection(upreg_genes_mycbp2)), len(upreg_genes_mtor), len(upreg_genes_mycbp2)

(13, 62, 1817)

In [20]:
len(upreg_genes_mtor.intersection(downreg_genes_mycbp2)), len(downreg_genes_mtor), len(upreg_genes_mycbp2)

(7, 1745, 1817)

In [21]:
len(downreg_genes_mtor.intersection(upreg_genes_mycbp2)), len(downreg_genes_mtor), len(upreg_genes_mycbp2)

(152, 1745, 1817)

In [30]:
pd.DataFrame(list(downreg_genes_mtor), columns=["MTOR_dn"]).to_csv("/home/neffra/zhang-ipy/Programs/downreg_genes_mtor.tsv", sep="\t", index=None)
pd.DataFrame(list(upreg_genes_mtor), columns=["MTOR_up"]).to_csv("/home/neffra/zhang-ipy/Programs/upreg_genes_mtor.tsv", sep="\t", index=None)
pd.DataFrame(list(downreg_genes_mycbp2), columns=["MYCBP2_dn"]).to_csv("/home/neffra/zhang-ipy/Programs/downreg_genes_mycbp2.tsv", sep="\t", index=None)
pd.DataFrame(list(upreg_genes_mycbp2), columns=["MYCBP2_up"]).to_csv("/home/neffra/zhang-ipy/Programs/upreg_genes_mycbp2.tsv", sep="\t", index=None)